<a href="https://colab.research.google.com/github/vivekkishore/machinelearning/blob/main/Agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

####Practice

In [ ]:
!pip install openai

In [ ]:
import openai
import os

from openai import AzureOpenAI

client=AzureOpenAI(azure_endpoint="https://devicemonitoring.openai.azure.com/",api_key="30b4ae5d18f74d90b9fcc563460d9f0a",api_version="2024-05-01-preview")

class agent:
    def __init__(self,sys=""):
        self.sys=sys
        self.messages=[]
        self.messages.append({"role":"system","content":self.sys})

    def __call__(self,user_input):
        self.messages.append({"role":"user","content":f"{user_input}"})
        result=self.execute()
        self.messages.append({"role":"assistant","content":result})
        return result

    def execute(self):
        response=client.chat.completions.create(model="gpt-4o",messages=self.messages,temperature=0)
        return response.choices[0].message.content


In [1]:
str="""Thought: I need to find the average weight of a Border Collie and a Scottish Terrier, then add them together to get the combined weight.
Action: average_dog_weight: Border Collie
PAUSE"""
lines=str.split("\n")

In [2]:
lines

['Thought: I need to find the average weight of a Border Collie and a Scottish Terrier, then add them together to get the combined weight.',
 'Action: average_dog_weight: Border Collie',
 'PAUSE']

In [ ]:
# prompt: match this pattern to pull action name and argument

import re

str="""Thought: I need to find the average weight of a Border Collie and a Scottish Terrier, then add them together to get the combined weight.
Action: average_dog_weight: Border Collie
PAUSE"""
lines=str.split("\n")

for line in lines:
  match = re.match(r"Action:\s*(\w+):\s*(.+)", line)
  if match:
    action_name = match.group(1)
    argument = match.group(2)
    print(f"Action Name: {action_name}")
    print(f"Argument: {argument}")


In [18]:
import re
action_re=re.compile(r"^Action: (.+):\s*(.+)$")
# action_re=re.compile(r"^Action:\s*[a-zA-Z]+:\s*[a-zA-Z]+$")

In [19]:
result=action_re.match(lines[1])
print(result)

<re.Match object; span=(0, 41), match='Action: average_dog_weight: Border Collie'>


In [20]:
result.groups()

('average_dog_weight', 'Border Collie')

####MultiThreading and multiprocessing

In [2]:
import threading
import time

def worker(n):
    print(f"Thread {n} starting")
    time.sleep(2)  # Simulating an I/O operation
    print(f"Thread {n} finished")

threads = []
for i in range(3):  # Creating 3 threads
    t = threading.Thread(target=worker, args=(i,))
    threads.append(t)
    t.start()

for t in threads:  # Wait for all threads to finish
    t.join()

print("All threads completed.")


Thread 0 starting
Thread 1 starting
Thread 2 starting
Thread 0 finished
Thread 1 finished
Thread 2 finished
All threads completed.


In [ ]:
import threading
import requests  # Or your preferred API library
import queue

def make_api_call(data_queue, results_queue):
    while True:
        try:
            data = data_queue.get_nowait()
        except queue.Empty:
            break  # Exit thread when queue is empty

        # Replace with your actual API call logic
        response = requests.post("your_api_endpoint", json=data)
        results_queue.put((data, response))  # Store data and response

        data_queue.task_done()

# Define data and number of threads
data_list = [{"key1": "value1"}, {"key2": "value2"}, ...]  # Your 100 data items
num_threads = 10  # Number of threads to use

# Initialize queues
data_queue = queue.Queue()
results_queue = queue.Queue()

# Populate data queue
for data in data_list:
    data_queue.put(data)

# Create and start threads
threads = []
for _ in range(num_threads):
    t = threading.Thread(target=make_api_call, args=(data_queue, results_queue))
    threads.append(t)
    t.start()

# Wait for all tasks to complete
data_queue.join()

# Retrieve results from results_queue
results = []
while not results_queue.empty():
    results.append(results_queue.get())

# Process the results
for data, response in results:
    # ... process data and response ...

print("All API calls completed.")

In [1]:
import multiprocessing

def square(n, result, index):
    """Function to compute square of a number and store it in shared list."""
    result[index] = n * n  # Store result in shared list

if __name__ == "__main__":
    numbers = [1, 2, 3, 4, 5]  # List of numbers
    manager = multiprocessing.Manager()  # Create a manager for shared memory
    result = manager.list([0] * len(numbers))  # Shared list to store results

    processes = []
    for i, num in enumerate(numbers):
        p = multiprocessing.Process(target=square, args=(num, result, i))
        processes.append(p)
        p.start()

    for p in processes:
        p.join()  # Wait for all processes to complete

    print("Squares:", list(result))  # Output the results


Squares: [1, 4, 9, 16, 25]
